# HiCoTiNe 2. Adapted from HiCoTiNe v3.3.1
## Concatenated hidden representations.
## No raw input
## The weight of each TN is inversely proportional to the number of TNs in its HiCo layer, relative to the HiCo layer with the most TNs

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from scipy import stats
from sklearn.metrics import accuracy_score
from keras import backend as K
import cv2

In [2]:
NUM_BATCH = 469
BATCH_SIZE = 128
SUB_REGION_SCALE = 14
MIN_SUB_REGION_SCALE = 12
MAX_SUB_REGION_SCALE = 16
NUM_TEST_STEP = 10
NUM_HICO_LAYER = 5
NUM_TN = [10, 8, 6, 4, 2]
NUM_CONNECTION = [0, 5, 4, 3, 2]
ALPHA = 0.1

#dataset specific parameters
NUM_CLASS = 10

## Data Pre-Processing

In [3]:
#load dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
#Generate cropped train image
X_train_cropped_list = []
y_train_cropped_list = []
#coordinate_list = []
for i in range(NUM_TN[0]):
  X_train_cropped_batch = []
  y_train_cropped_batch = []
  for j in range(NUM_BATCH):
    X_train_cropped = []
    y_train_cropped = []
    sub_region_scale = random.randint(MIN_SUB_REGION_SCALE, MAX_SUB_REGION_SCALE)
    x = random.randint(0, X_train.shape[1] - sub_region_scale)
    y = random.randint(0, X_train.shape[1] - sub_region_scale)
    
    if (j == NUM_BATCH - 1) and ((len(X_train) - j*BATCH_SIZE) % BATCH_SIZE):
      for k in range(len(X_train) - j*BATCH_SIZE):
        image_cropped = []
        for row in range(y, y + sub_region_scale):
          image_cropped_row = []
          for column in range(x, x + sub_region_scale):
            image_cropped_row.append(X_train[(j*BATCH_SIZE)+k][row][column])
          image_cropped.append(image_cropped_row)
        X_train_cropped.append(image_cropped)
        y_train_cropped.append(y_train[(j*BATCH_SIZE)+k])
      
      X_train_cropped = np.array(X_train_cropped)
      y_train_cropped = np.array(y_train_cropped)
      X_train_cropped_batch.append(X_train_cropped)
      y_train_cropped_batch.append(y_train_cropped)
      break

    for k in range(BATCH_SIZE):
      image_cropped = []
      for row in range(y, y + sub_region_scale):
        image_cropped_row = []
        for column in range(x, x + sub_region_scale):
          image_cropped_row.append(X_train[(j*BATCH_SIZE)+k][row][column])
        image_cropped.append(image_cropped_row)
      X_train_cropped.append(image_cropped)
      y_train_cropped.append(y_train[(j*BATCH_SIZE)+k])

    X_train_cropped = np.array(X_train_cropped)
    y_train_cropped = np.array(y_train_cropped)
    X_train_cropped_batch.append(X_train_cropped)
    y_train_cropped_batch.append(y_train_cropped)
  X_train_cropped_list.append(X_train_cropped_batch)
  y_train_cropped_list.append(y_train_cropped_batch)

In [5]:
#Generate cropped test image
X_test_cropped_list = []
for i in range(NUM_TN[0] * NUM_TEST_STEP):
  X_test_cropped = []
  sub_region_scale = random.randint(MIN_SUB_REGION_SCALE, MAX_SUB_REGION_SCALE)
  x = random.randint(0, X_test.shape[1] - sub_region_scale)
  y = random.randint(0, X_test.shape[1] - sub_region_scale)
  for j in range(X_test.shape[0]):
    image_cropped = []
    for row in range(y, y + sub_region_scale):
      image_cropped_row = []
      for column in range(x, x + sub_region_scale):
        image_cropped_row.append(X_test[j][row][column])
      image_cropped.append(image_cropped_row)
    X_test_cropped.append(image_cropped)

  X_test_cropped = np.array(X_test_cropped)
  X_test_cropped_list.append(X_test_cropped)

In [6]:
#rescaling dataset
for i in range(NUM_TN[0]):
  for j in range(NUM_BATCH):
    temp = []
    for k in range (len(X_train_cropped_list[i][j])):
      temp.append(cv2.resize(X_train_cropped_list[i][j][k], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
    temp = np.array(temp)
    X_train_cropped_list[i][j] = temp

for i in range(NUM_TN[0] * NUM_TEST_STEP):
  temp = []
  for j in range(X_test.shape[0]):
    temp.append(cv2.resize(X_test_cropped_list[i][j], (SUB_REGION_SCALE, SUB_REGION_SCALE)))
  temp = np.array(temp)
  X_test_cropped_list[i] = temp

In [7]:
#normalizing dataset
for i in range(NUM_TN[0]):
  for j in range(NUM_BATCH):
    X_train_cropped_list[i][j] = X_train_cropped_list[i][j]/255

for i in range(NUM_TN[0] * NUM_TEST_STEP):
  X_test_cropped_list[i] = X_test_cropped_list[i]/255

In [8]:
#transform to vector
for i in range(NUM_TN[0]):
  for j in range(NUM_BATCH):
    X_train_cropped_list[i][j] = X_train_cropped_list[i][j].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE))

for i in range(NUM_TN[0] * NUM_TEST_STEP):
  X_test_cropped_list[i] = X_test_cropped_list[i].reshape((-1, SUB_REGION_SCALE*SUB_REGION_SCALE))

In [9]:
#one-hot encoding
y_train_one_hot = []
for i in range(NUM_TN[0]):
  temp = []
  for j in range(NUM_BATCH):
    temp.append(to_categorical(y_train_cropped_list[i][j], num_classes=NUM_CLASS))
  y_train_one_hot.append(temp)

y_test_one_hot = to_categorical(y_test)

## HiCo Layer 1

In [10]:
#build ANN model
ensemble = []
for i in range(NUM_TN[0]):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append((model, i))

In [11]:
for i in range(NUM_TN[0]):
  print('Model %d' %i)
  for j in range(5):
    print('Epoch %d' %j)
    for k in range(NUM_BATCH):
      ensemble[i][0].train_on_batch(X_train_cropped_list[i][k], y_train_one_hot[i][k])

Model 0
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 1
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 2
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 3
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 4
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 5
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 6
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 7
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 8
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Model 9
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [12]:
# Predict on test image
y_pred_list = []
for i in range(NUM_TN[0]):
  y_pred = np.argmax(ensemble[i][0].predict(X_test_cropped_list[i]), axis=1)
  y_pred_list.append(y_pred)
    
# Ensemble voting (mode)
y_pred_list = np.array(y_pred_list)
y_pred_list = np.transpose(y_pred_list, (1, 0))
y_pred_list = stats.mode(y_pred_list, axis=1)[0]
y_pred_list = np.squeeze(y_pred_list)

In [13]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.6895

In [14]:
hico_layers = []
hico_layers.append(ensemble)

In [15]:
del hico_layers[1:]

## HiCo Layer 2+

In [16]:
X_train_cropped_list_combined = []
for i in range(NUM_TN[0]):
  X_train_cropped_list_combined.append(np.concatenate(X_train_cropped_list[i], axis=0))

y_train_one_hot_combined = to_categorical(y_train)

In [17]:
# Function to get input of layer i-1
def get_previous_layer_input(hico_layers, layer, tn, train_image):
  input = []
    
  if layer == 0:
    previous_layer_input = train_image[tn[1]]
    return previous_layer_input

  elif layer > 0:
    for i in range(NUM_CONNECTION[layer]):
      get_input = (K.function(hico_layers[layer-1][tn[1][i]][0].layers[0].input, hico_layers[layer-1][tn[1][i]][0].layers[1].output))
      input.append(get_input(get_previous_layer_input(hico_layers, layer-1, hico_layers[layer-1][tn[1][i]], train_image)))
      
    input = np.array(input)
    input = np.concatenate(input, axis=1)
    previous_layer_input = input
    return previous_layer_input

In [18]:
for i in range(1, NUM_HICO_LAYER):
  print('Layer %d' %i)
  ensemble = []
  X_train_input_list = []
  X_test_input_list = []

  for j in range(NUM_TN[i]):
    # Build model of HiCo layer i
    connection = tuple(random.sample(range(len(hico_layers[i-1])), k=NUM_CONNECTION[i]))
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=64*NUM_CONNECTION[i]))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(NUM_CLASS, activation = 'softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    ensemble.append((model, connection))
  hico_layers.append(ensemble)
  print('HICO LENGTH')
  print(len(hico_layers))

  for j in range(NUM_TN[i]):
    # Get train hidden representation from HiCo layer i-1
    X_train_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_train_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_train_cropped_list_combined)))
    X_train_input = np.array(X_train_input)
    X_train_input = np.concatenate(X_train_input, axis=1)
    X_train_input_list.append(X_train_input)

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.concatenate(X_test_input, axis=1)
    X_test_input_list.append(X_test_input)

  #train model of HiCo layer i
  for j in range(NUM_TN[i]):
    print('Model %d' %j)
    hico_layers[i][j][0].fit(X_train_input_list[j], y_train_one_hot_combined, validation_data=(X_test_input_list[j], y_test_one_hot), epochs=5)

Layer 1
HICO LENGTH
2
Model 0
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9739 - accuracy: 0.6307 - val_loss: 0.7725 - val_accuracy: 0.7012
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.7694 - accuracy: 0.7018 - val_loss: 0.7306 - val_accuracy: 0.7210
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.7431 - accuracy: 0.7089 - val_loss: 0.7261 - val_accuracy: 0.7203
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.7177 - accuracy: 0.7202 - val_loss: 0.7485 - val_accuracy: 0.7083
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.7093 - accuracy: 0.7211 - val_loss: 0.7124 - val_accuracy: 0.7279
Model 1
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.9983 - accuracy: 0.6186 - val_loss: 0.7667 - val_accuracy: 0.7055
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.7730 - accuracy: 0.6965 - v

## Model Evaluation

In [19]:
# Predict on test image
y_pred_list_2 = []
for h in range(NUM_TEST_STEP):
  y_pred_list = []
  for j in range(NUM_TN[0]):
    y_pred = hico_layers[0][j][0].predict(X_test_cropped_list[(h*NUM_TN[0])+j])
    y_pred = y_pred * ((1 + ALPHA) - (NUM_TN[0] / max(NUM_TN)))
    y_pred_list.append(y_pred)

  for i in range(1, NUM_HICO_LAYER):
    for j in range(NUM_TN[i]):

      # Get test hidden representation from HiCo layer i-1
      X_test_input = []
      for k in range(NUM_CONNECTION[i]):
        get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[1].output))
        X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list[h*NUM_TN[0]:(h+1)*NUM_TN[0]])))
      X_test_input = np.array(X_test_input)
      X_test_input = np.concatenate(X_test_input, axis=1)
      y_pred = hico_layers[i][j][0].predict(X_test_input)
      y_pred = y_pred * ((1 + ALPHA) - (NUM_TN[i] / max(NUM_TN)))
      y_pred_list.append(y_pred)

  # HiCo voting
  y_pred_list = np.array(y_pred_list)
  y_pred_list = np.transpose(y_pred_list, (1, 0, 2))
  y_pred_list = np.sum(y_pred_list, axis=1)
  y_pred_list_2.append(y_pred_list)

y_pred_list_2 = np.array(y_pred_list_2)
y_pred_list_2 = np.transpose(y_pred_list_2, (1, 0, 2))
y_pred_list_2 = np.argmax(np.sum(y_pred_list_2, axis=1), axis=1)

In [20]:
accuracy = accuracy_score(y_test, y_pred_list_2)
accuracy

0.7882